<a href="https://colab.research.google.com/github/FraGoTe/Analisis-Estadistico-Textos/blob/master/PreProcesamientoTexto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Operaciones básicas de preprocesamiento de texto

.



Importar bibliotecas

In [0]:
#!/usr/bin/python3

import nltk, re
from nltk import CFG

from nltk.chunk.regexp import *
from nltk.parse.stanford import StanfordParser
from nltk.corpus import udhr
from nltk.stem.snowball import SnowballStemmer

#esta biblioteca sirve para leer un directorio con archivos
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import XMLCorpusReader

from nltk.corpus import webtext;
from nltk.corpus import stopwords;
from nltk.collocations import BigramCollocationFinder;
from nltk.collocations import TrigramCollocationFinder;
from nltk.metrics import BigramAssocMeasures;
from nltk.metrics import TrigramAssocMeasures;

from pathlib import Path


.

.

.

descargar algunas bibliotecas por si no existen. Esto suele requerirse cada vez que se referesca el entorno colab

In [2]:
nltk.download('conll2000')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

nltk.download('treebank')

[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

.

.

.

Funciones para cargar documentos de un directorio

In [0]:
def cargaDirectorio(directorio):
  #se puede indicar un filtro de archivos, en este caso cargar todos (.*)
  wordlists = PlaintextCorpusReader(directorio, '.*') 
  wordlists.fileids()

#  wordlists.raw(fileid)
  return wordlists


#carga un solo archivo de acuerdo a su ruta
def cargarArchivoTexto(ruta):
  f=open(ruta, 'r', encoding='utf8')
  text=f.read()
  return text



def downloadDataset():
  !rm -R datasets
  !mkdir -p datasets
  #!wget https://github.com/githila/data/raw/master/ingles-corpus.zip   -P datasets
  !wget https://github.com/githila/data/raw/master/espanol-corpus.zip   -P datasets
  
  #!unzip datasets/ingles-corpus.zip 
  #!rm datasets/ingles-corpus.zip

  !unzip datasets/espanol-corpus.zip 
  !rm datasets/espanol-corpus.zip

.

.

.

Funciones para preprocesar el texto

In [0]:


 #Divide un texto en tokens
def tokenizacion(text):
  tokens=nltk.word_tokenize(text, "spanish")
  print(tokens,"\n")


  #Obtiene las palabras raiz en una sentencia
def stemming(texto):
  stemmer = SnowballStemmer("spanish")
  tokens=nltk.word_tokenize(texto, "spanish")
  singles = [stemmer.stem(word) for word in tokens]
  print(singles)


#segmentación de sentencias. Divide un texto en sentencias
def segmentacion_sentencias(texto):
  sentences = nltk.sent_tokenize(texto, "spanish")
  print("Sentencias")
  print(sentences,"\n")


#etiquetado gramatical de texto, primero hay que separar en tokens
def pos_tagging(texto):
  tokens=nltk.word_tokenize(texto, "spanish")
  tags=nltk.pos_tag(tokens,lang="spanish")
  print(tags)


#imprime las etiquetas chunk con sus entidades
def ner(texto): 
  #Segmentación de sentencias
  sentences = nltk.sent_tokenize(texto, "spanish")

  #tokenizacion
  tokenized_sentences = [nltk.word_tokenize(sentence, "spanish") for sentence in sentences]
  
  #pos tagging
  tagged_sentences = [nltk.pos_tag(sentence,lang="spanish") for sentence in tokenized_sentences]
  
  #chunking agrupa elementos en frases constituyentes (sin el mismo nivel de expresividad que un arbol)
  for sent in tagged_sentences:
    print(nltk.ne_chunk(sent, "spanish"))


#funciona para inglés, devuelve el arbol de constituyentes de una sentencia
def parser():
  from nltk.corpus import treebank
  from nltk import PCFG, ViterbiParser

  productions = []
  S = nltk.Nonterminal('S')
  for f in treebank.fileids():
	  for tree in treebank.parsed_sents(f):
		  productions += tree.productions()
  grammar = nltk.induce_pcfg(S,productions)
  for p in grammar.productions()[1:15]:
	  print(p)
  parser = ViterbiParser(grammar)
  parse,=parser.parse(['it','is','a','small','group','of','workers','and','researchers'])
  print(parse,"\n")



.

.

.

Encontrar bigramas y trigramas

In [0]:

#para encontrar bigramas
def find_bigram(texto): # Función para encontrar bigramas
  tokens=nltk.word_tokenize(texto, "spanish")
  words = [w.lower() for w in tokens]; # Obtiene las palabras de un corpus

  bcf = BigramCollocationFinder.from_words(words); # Obtiene un objeto bigrama
  bcf = scape_stopwords(bcf); # Se limpia el conjunto de palabras; se quitan las palabras vacías
  scored = bcf.score_ngrams(BigramAssocMeasures.raw_freq); # Se obtiene los bigramas por puntaje
  print('Scored')
  for pair in range(0,10): # Se imprime solo los 10 primeros bigramas
    print(scored[pair]);

  likelihood_ratio = bcf.nbest(BigramAssocMeasures.likelihood_ratio, 10); # Se obtienen los bigramas por probabilidad
  print('\nLikelihood ratio')
  for pair in likelihood_ratio: # Se imprime solo los 10 primeros
    print(pair);

  chi_sq = bcf.nbest(BigramAssocMeasures.chi_sq, 10); # Se obtienen los bigramas utilizando chi-squad
  print('\nChi-squared')
  for pair in chi_sq: # Se imprime solo los 10 primeros
    print(pair);

  pmi = bcf.nbest(BigramAssocMeasures.pmi, 10); 
  # Se obtiene bigramas por PMI
  print('\nPoint-wise mutual information');
  for pair in pmi: # Se imprime los 10 primeros
    print(pair);


## Función para quitar stopwords
def scape_stopwords( bcf): 
  stopset = set(stopwords.words('spanish')); # Lenguaje
  filter_stops = lambda w: len(w) < 3 or w in stopset;
  bcf.apply_word_filter(filter_stops);
  return bcf;


def find_triplets(texto): # Función para encontrar trigramas
		# Se aplica el mismo mecaniscos que con los bigramas

  tokens=nltk.word_tokenize(texto, "spanish")
  words = [w.lower() for w in tokens]; # Obtiene las palabras de un corpus
  tcf = TrigramCollocationFinder.from_words(words); # Obtener el objeto de trigramas
  tcf = scape_stopwords(tcf); # Quita las palabras vacías
  tcf.apply_freq_filter(3); # Se aplica el filtro para trigramas
  scored = tcf.score_ngrams(TrigramAssocMeasures.raw_freq); # Trigramas por puntaje
  print('Scored')
  numTrigrams=len(scored)

  for triplets in range(0,numTrigrams): # Se imprimen 10 primeros
    print(scored[triplets]);

  likelihood_ratio = tcf.nbest(TrigramAssocMeasures.likelihood_ratio, numTrigrams); # Trigramas por probabilidad
  print('\nLikelihood ratio')
  for triplets in likelihood_ratio: # Se imprimen 10 primeros
    print(triplets);

  chi_sq = tcf.nbest(TrigramAssocMeasures.chi_sq, numTrigrams); # Trigramas por chi-squad
  print('\nChi-squared')
  for triplets in chi_sq: # Se imprimen 10 primeros
    print(triplets);

  pmi = tcf.nbest(TrigramAssocMeasures.pmi, numTrigrams); # Trigramas por PMI
  print('\nPoint-wise mutual information');
  for triplets in pmi: # Se imprimen 10 primeros
    print(triplets);



.

.

.

Método principal

In [6]:
p = Path('/content/corpus') #('/content/datasets')
if not p.exists():
  downloadDataset() ##ejecutar una sola vez para descargar datasets


#obtener archivos de un directorio/dataset
directorio= cargaDirectorio('/content/corpus')

texto=directorio.raw() #carga todo el texto text  / allText

rm: cannot remove 'datasets': No such file or directory
--2019-11-25 22:09:16--  https://github.com/githila/data/raw/master/espanol-corpus.zip
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/githila/data/master/espanol-corpus.zip [following]
--2019-11-25 22:09:16--  https://raw.githubusercontent.com/githila/data/master/espanol-corpus.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167929 (164K) [application/zip]
Saving to: ‘datasets/espanol-corpus.zip’

espanol-corpus.zip  100%[===================>] 163.99K  --.-KB/s    in 0.01s   

2019-11-25 22:09:17 (14.2 MB/s) - ‘datasets/espanol-corpus.zip’ 

## Tokenizando un solo archivo

In [7]:

texto=directorio.raw('deportes01.txt') #carga un solo archivo

#tokenizacion
tokenizacion(texto) #texto


['España', 'gana', 'el', 'Mundial', 'sub-17', 'de', 'fútbol', 'femenino', 'Las', 'chicas', 'de', 'Toña', 'Is', 'derrotan', 'a', 'México', 'en', 'la', 'final', '(', '2-1', ')', 'y', 'se', 'convierten', 'en', 'el', 'primer', 'combinado', 'femenino', 'español', 'campeón', 'del', 'mundo', '.', 'Al', 'fútbol', 'femenino', 'español', 'le', 'faltaba', 'un', 'Mundial', ',', 'y', 'las', 'chicas', 'de', 'la', 'selección', 'sub-17', 'entrenadas', 'por', 'Toña', 'Is', 'lo', 'lograron', 'este', 'sábado', 'ante', 'México', 'en', 'la', 'final', 'en', 'Montevideo', '(', '2-1', ')', '.', 'España', 'solo', 'tiene', 'otras', 'dos', 'Copas', 'del', 'Mundo', ',', 'ambas', 'en', 'categoría', 'masculina', ':', 'la', 'lograda', 'por', 'la', 'sub-20', 'en', 'Nigeria', 'en', '1999', 'y', 'la', 'que', 'llegó', 'en', '2010', 'en', 'Sudáfrica', 'por', 'algunos', 'de', 'aquellos', 'jugadores', 'que', 'ya', 'no', 'pensaban', 'en', 'el', 'fallo', 'de', 'Cardeñosa', ',', 'en', 'el', 'codazo', 'de', 'Tassotti', 'ni', '

## Stemming

In [8]:
##stemming
stemming(texto)


['españ', 'gan', 'el', 'mundial', 'sub-17', 'de', 'futbol', 'femenin', 'las', 'chic', 'de', 'toñ', 'is', 'derrot', 'a', 'mexic', 'en', 'la', 'final', '(', '2-1', ')', 'y', 'se', 'conviert', 'en', 'el', 'prim', 'combin', 'femenin', 'español', 'campeon', 'del', 'mund', '.', 'al', 'futbol', 'femenin', 'español', 'le', 'falt', 'un', 'mundial', ',', 'y', 'las', 'chic', 'de', 'la', 'seleccion', 'sub-17', 'entren', 'por', 'toñ', 'is', 'lo', 'logr', 'este', 'sab', 'ante', 'mexic', 'en', 'la', 'final', 'en', 'montevide', '(', '2-1', ')', '.', 'españ', 'sol', 'tien', 'otras', 'dos', 'cop', 'del', 'mund', ',', 'ambas', 'en', 'categor', 'masculin', ':', 'la', 'logr', 'por', 'la', 'sub-20', 'en', 'nigeri', 'en', '1999', 'y', 'la', 'que', 'lleg', 'en', '2010', 'en', 'sudafr', 'por', 'algun', 'de', 'aquell', 'jugador', 'que', 'ya', 'no', 'pens', 'en', 'el', 'fall', 'de', 'cardeñ', ',', 'en', 'el', 'codaz', 'de', 'tassotti', 'ni', 'en', 'el', 'rest', 'de', 'maldicion', '.', 'si', 'algo', 'demostr', 'l

In [9]:

##segmentación de sentencias
segmentacion_sentencias(texto)


Sentencias
['España gana el Mundial sub-17 de fútbol femenino\nLas chicas de Toña Is derrotan a México en la final (2-1) y se convierten en el primer combinado femenino español campeón del mundo.', 'Al fútbol femenino español le faltaba un Mundial, y las chicas de la selección sub-17 entrenadas por Toña Is lo lograron este sábado ante México en la final en Montevideo (2-1).', 'España solo tiene otras dos Copas del Mundo, ambas en categoría masculina: la lograda por la sub-20 en Nigeria en 1999 y la que llegó en 2010 en Sudáfrica por algunos de aquellos jugadores que ya no pensaban en el fallo de Cardeñosa, en el codazo de Tassotti ni en el resto de maldiciones.', 'Si algo demostraron las jugadoras de Is durante este Mundial sub-17 y especialmente durante la final es que participan también de ese estado de ánimo: como los grandes equipos, juegan con la alegría de saber que no deben nada a nadie y con la intrepidez que exige abrir camino.', 'Las señales que emiten permiten imaginarlas di

In [10]:

##POS tagging
pos_tagging(texto)


[('España', 'NNP'), ('gana', 'NN'), ('el', 'VBZ'), ('Mundial', 'JJ'), ('sub-17', 'JJ'), ('de', 'FW'), ('fútbol', 'NN'), ('femenino', 'NN'), ('Las', 'NNP'), ('chicas', 'NN'), ('de', 'FW'), ('Toña', 'NNP'), ('Is', 'NNP'), ('derrotan', 'VBZ'), ('a', 'DT'), ('México', 'NNP'), ('en', 'NN'), ('la', 'FW'), ('final', 'JJ'), ('(', '('), ('2-1', 'JJ'), (')', ')'), ('y', 'FW'), ('se', 'FW'), ('convierten', 'VBP'), ('en', 'FW'), ('el', 'FW'), ('primer', 'NN'), ('combinado', 'NN'), ('femenino', 'NN'), ('español', 'VBP'), ('campeón', 'NN'), ('del', 'FW'), ('mundo', 'NN'), ('.', '.'), ('Al', 'NNP'), ('fútbol', 'NN'), ('femenino', 'NN'), ('español', 'VBP'), ('le', 'JJ'), ('faltaba', 'NN'), ('un', 'JJ'), ('Mundial', 'NNP'), (',', ','), ('y', 'NNP'), ('las', 'VBZ'), ('chicas', 'FW'), ('de', 'FW'), ('la', 'FW'), ('selección', 'FW'), ('sub-17', 'JJ'), ('entrenadas', 'NNS'), ('por', 'JJ'), ('Toña', 'NNP'), ('Is', 'NNP'), ('lo', 'JJ'), ('lograron', 'NN'), ('este', 'NN'), ('sábado', 'NN'), ('ante', 'NN'), ('

In [13]:

##reconocimiento de entidades nombradas
ner(directorio.raw('deportes01.txt'))


(S
  (NE España/NNP)
  gana/NN
  el/VBZ
  (NE Mundial/JJ)
  sub-17/JJ
  de/FW
  fútbol/NN
  femenino/NN
  Las/NNP
  chicas/NN
  de/FW
  Toña/NNP
  Is/NNP
  derrotan/VBZ
  a/DT
  (NE México/NNP)
  en/NN
  la/FW
  final/JJ
  (/(
  2-1/JJ
  )/)
  y/FW
  se/FW
  convierten/VBP
  en/FW
  el/FW
  primer/NN
  combinado/NN
  femenino/NN
  español/VBP
  campeón/NN
  del/FW
  mundo/NN
  ./.)
(S
  (NE Al/NNP)
  fútbol/NN
  femenino/NN
  español/VBP
  le/JJ
  faltaba/NN
  un/JJ
  Mundial/NNP
  ,/,
  y/NNP
  las/VBZ
  chicas/FW
  de/FW
  la/FW
  selección/FW
  sub-17/JJ
  entrenadas/NNS
  por/JJ
  Toña/NNP
  Is/NNP
  lo/JJ
  lograron/NN
  este/NN
  sábado/NN
  ante/NN
  (NE México/NNP)
  en/VBZ
  la/FW
  final/JJ
  en/NN
  Montevideo/NNP
  (/(
  2-1/JJ
  )/)
  ./.)
(S
  (NE España/NNP)
  solo/NN
  tiene/NN
  otras/VBP
  dos/JJ
  Copas/NNP
  del/FW
  Mundo/NNP
  ,/,
  ambas/NN
  en/FW
  categoría/NN
  masculina/NN
  :/:
  la/NN
  lograda/FW
  por/NN
  la/FW
  sub-20/JJ
  en/FW
  Nigeria/NNP
  en/IN


In [14]:
##collocations
##bigramas
find_bigram(texto)


Scored
(('cata', 'coll'), 0.002129925452609159)
(('claudia', 'pina'), 0.002129925452609159)
(('eva', 'navarro'), 0.002129925452609159)
(('femenino', 'español'), 0.002129925452609159)
(('fútbol', 'femenino'), 0.002129925452609159)
(('mundial', 'sub-17'), 0.002129925452609159)
(('segundo', 'gol'), 0.002129925452609159)
(('abrir', 'camino'), 0.0010649627263045794)
(('ahora', 'parecían'), 0.0010649627263045794)
(('algún', 'balón'), 0.0010649627263045794)

Likelihood ratio
('cata', 'coll')
('claudia', 'pina')
('eva', 'navarro')
('segundo', 'gol')
('femenino', 'español')
('fútbol', 'femenino')
('mundial', 'sub-17')
('ahora', 'parecían')
('andan', 'aún')
('anotó', 'casi')

Chi-squared
('ahora', 'parecían')
('andan', 'aún')
('anotó', 'casi')
('aquellos', 'jugadores')
('años', 'controló')
('botas', 'cargadas')
('buscar', 'rápido')
('castro', 'aprovechó')
('cata', 'coll')
('clave', 'funcionaron')

Point-wise mutual information
('ahora', 'parecían')
('andan', 'aún')
('anotó', 'casi')
('aquellos',

In [16]:

##trigramas
find_triplets(texto)


Scored

Likelihood ratio

Chi-squared

Point-wise mutual information


In [17]:

##solo para textos en ingles
 parser()

NP-SBJ -> NP , ADJP , [0.000392567]
NP -> NNP NNP [0.0309391]
NNP -> 'Pierre' [0.00010627]
NNP -> 'Vinken' [0.00021254]
, -> ',' [0.999795]
ADJP -> NP JJ [0.014556]
NP -> CD NNS [0.0110015]
CD -> '61' [0.00141004]
NNS -> 'years' [0.0190177]
JJ -> 'old' [0.00411382]
VP -> MD VP [0.0523088]
MD -> 'will' [0.30205]
VP -> VB NP PP-CLR NP-TMP [0.000137836]
VB -> 'join' [0.00156617]
(S
  (NP-SBJ (PRP it))
  (VP
    (VBZ is)
    (NP-PRD
      (NP (DT a) (JJ small) (NN group))
      (PP
        (IN of)
        (NP (NNS workers) (CC and) (NNS researchers)))))) (p=2.64379e-21) 



## Descargando Archivos en PDF / doc / ppt

In [0]:
def downloadAdFiles():
  !rm -R datasets
  !mkdir -p datasets
  #!wget https://github.com/githila/data/raw/master/ingles-corpus.zip   -P datasets
  !wget https://media.ldscdn.org/pdf/lds-scriptures/holy-bible/holy-bible-spa.pdf  -P files
  !wget http://www.consulado.pe/Documents/Solicitud-Consulado.docx  -P files
  !wget http://cyberprofesor.pbworks.com/f/Manual%2BPHP%2BB%C3%A1sico.ppt -P files
  

In [64]:
downloadAdFiles()

--2019-11-25 22:55:43--  https://media.ldscdn.org/pdf/lds-scriptures/holy-bible/holy-bible-spa.pdf
Resolving media.ldscdn.org (media.ldscdn.org)... 92.122.252.47
Connecting to media.ldscdn.org (media.ldscdn.org)|92.122.252.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12838859 (12M) [application/pdf]
Saving to: ‘files/holy-bible-spa.pdf.1’

holy-bible-spa.pdf. 100%[===================>]  12.24M  --.-KB/s    in 0.1s    

2019-11-25 22:55:43 (90.6 MB/s) - ‘files/holy-bible-spa.pdf.1’ saved [12838859/12838859]

--2019-11-25 22:55:45--  http://www.consulado.pe/Documents/Solicitud-Consulado.docx
Resolving www.consulado.pe (www.consulado.pe)... 190.116.24.58
Connecting to www.consulado.pe (www.consulado.pe)|190.116.24.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22418 (22K) [application/vnd.ms-word.document.12]
Saving to: ‘files/Solicitud-Consulado.docx’

Solicitud-Consulado 100%[===================>]  21.89K   115KB/s    in 0.2s 

## Leyendo Archivo PDF

In [26]:
!pip install PyPDF2 

     |████████████████████████████████| 81kB 4.2MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61085 sha256=e1300af2a08a04708ff668beeb04e480e77b451992ceb2298a8cf9892d38ae6e
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [50]:
import PyPDF2 
  
pdfFileObj = open('files/holy-bible-spa.pdf', 'rb') 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
print(pdfReader.numPages) 


2108


In [54]:
pageObj = pdfReader.getPage(40) 
print(pageObj.extractText())

laBilbliaPage40 = pageObj.extractText()

8 Tomó también cuajada y leche, 
y el becerro que había preparado, 

y lo puso delante de ellos; y él 

estaba junto a ellos debajo del 

árbol, y comieron.
9 Y le dijeron: ¿Dónde está Sara, 
tu esposa? Y él respondió: Aquí 

en la tienda.10 Entonces dijo: De cierto vol-veré a ti según el tiempo de la 

vida, y he aquí, Sara, tu esposa, 

tendrá un a hijo. Y Sara escuchaba 
a la puerta de la tienda, que es-
taba detrás de él.11 Y Abraham y Sara eran vie-
jos, entrados en años; a Sara le 

había cesado ya la costumbre de 

las mujeres.
12 Se rió, pues, Sara para sí, 
diciendo: ¿Después que he en-
vejecido tendré deleite, siendo 

también mi señor ya viejo?
13 Entonces Jehová dijo a Abra-
ham: ¿Por qué se ha reído Sara, 

diciendo: Será cierto que he de 

dar a luz siendo ya vieja?14 ¿Hay para Dios alguna a cosa difícil? Al tiempo señalado vol-

veré a ti, según el tiempo de la 

vida, y Sara tendrá un hijo.15 Entonces Sara lo negó, por-
que tuvo miedo, diciendo: No me 

reí. Y él dijo: 

In [61]:
!pip install win32com.client

ERROR: Could not find a version that satisfies the requirement win32com.client (from versions: none)
ERROR: No matching distribution found for win32com.client


In [70]:
import docx2txt

solicitud = docx2txt.process("files/Solicitud-Consulado.docx")
print(solicitud)


MODELO DE SOLICITUD





SUMILLA: Solicito…. (Precise el tipo de asistencia que necesita)

----------------------------------------  



Señor Cónsul General del Perú en………….. (Señale la ciudad y el país donde se localiza el asunto)





		Yo, … (nombres y apellidos) ….,  identificado/a con DNI Nº ………, domiciliado/a en … (Av., Calle, Nº, Urbanización, Distrito, Provincia, Departamento)., con teléfono ( fijo y/o móvil) Nº ….., ante usted me presento y expongo:  



		Que,…. (Describa, explique el asunto en forma clara, proporcionando los mayores datos posibles, e indique las razones por las que solicita esta asistencia) 



		Por tal motivo, solicito apoyo del Consulado para… (Precise el tipo de asistencia que necesita). 



   		Por lo expuesto: 

		Ruego a usted acceder a lo solicitado por ser de justicia.

	



					Lugar y fecha   





					______________________________________ 

						           (Firma del/la recurrente)  





----------------------------------------------------

## Buscando Acronimos

In [96]:
import re

texto = 'Mi amigo viene de U.S.A.'

pattern = r'(?:[A-Z]\.)+'
print(re.findall(pattern, texto))


['U.S.A.']


# Buscando Fechas, detectando la posicion y elminandolas



In [105]:
mc = 'mi cumpleanos es el 30/11/1991'
match = re.search(r'\d{2}/\d{2}/\d{4}', mc)

print('La posicion de la fecha es ' +  str(match.start()))
print('\nEl texto sin fecha es: \n ' + mc.replace(str(match.group()), ''))

La posicion de la fecha es 20

El texto sin fecha es: 
 mi cumpleanos es el 


In [106]:
find_bigram(laBilbliaPage40)

Scored
(('cincuenta', 'justos'), 0.0033444816053511705)
(('entonces', 'jehová'), 0.0033444816053511705)
(('hacia', 'sodoma'), 0.0033444816053511705)
(('jehová', 'dijo'), 0.0033444816053511705)
(('abra-', 'ham'), 0.0016722408026755853)
(('abraham', 'iba'), 0.0016722408026755853)
(('abraham—la', 'descendencia'), 0.0016722408026755853)
(('acercó', 'abraham'), 0.0016722408026755853)
(('aún', 'delante'), 0.0016722408026755853)
(('ca-', 'mino'), 0.0016722408026755853)

Likelihood ratio
('cincuenta', 'justos')
('hacia', 'sodoma')
('abra-', 'ham')
('abraham—la', 'descendencia')
('ca-', 'mino')
('casa', 'después')
('cosa', 'difícil')
('descenderé', 'ahora')
('en-', 'vejecido')
('es-', 'taba')

Chi-squared
('abra-', 'ham')
('abraham—la', 'descendencia')
('ca-', 'mino')
('casa', 'después')
('cincuenta', 'justos')
('cosa', 'difícil')
('descenderé', 'ahora')
('en-', 'vejecido')
('es-', 'taba')
('familia—las', 'responsabilidades')

Point-wise mutual information
('abra-', 'ham')
('abraham—la', 'desce